In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pro_data

# 企业变更数据的对应特征(2alter.csv)
    
    1. 变更的事件的总个数, ALTERNO_SUM
    2. 变更的事件的类型的种类数, ALTERNO_CNT
    3. 变更的事件的类型的种类数 / 所有事件的总总的类别数, ALTERNO_CNT_RATE
    4. 第一次变更时间, FIRST_CHANGE_TIME
    5. 第一次变更距离企业注册时间的差（单位：月）,不存在变更事件的用0填充, FIRST_CHANGE_TIME_DIFF
    6. 最后一次变更时间, END_CHANGE_TIME
    7. 最后一次变更时间距离企业注册时间的差值，END_CHANGE_TIME_DIFF
    8. 最后一次变更距离第一次事件变更时间的差（单位：月）,不存在变更事件的用0填充, END_FIRST_CHANGE_TIME_DIFF

    9. 变更的事件是否发生了前后的值得变化（1/0）, IS_CHANGE
    10. 第一次变更事件的类型，CHANGE_FIRST_ALTERNO
    11. 最后一次变更事件的类型，CHANGE_END_ALTERNO
    12. 最多的变更事件的类型，CHANGE_ALTERNO_MUCNID
    
    新加特征
    -------------------------------------------------------------------------------------------------------
    13. 平均每几个月变更一次事件，CHANGE_PRE_CNT = ALTERNO_SUM / END_CHANGE_TIME_DIFF
    14. 每个企业变更事件的个数占所有企业变更事件平均值的比例， CHANGE_CNT_RATE = ALTERNO_SUM / AVG 
    15. 企业对应大类HY的平均变更次数，CHANGE_HY_CNT_AVG
    16. 企业对应大类HY的平均变更次数占所有变更时间的个数的平均值的比例，CHANGE_HY_CNT_ALL_AVG




In [2]:
df_all = pd.read_csv("../data/alldata/df_data1.csv")
df_change = pd.read_csv("../data/public/2alter.csv")

In [3]:
df_all['ETYPE'].value_counts()

7     206500
6      18375
8      11114
17      8798
5       3890
16      3543
13      1597
3        843
2        339
1        111
4         20
Name: ETYPE, dtype: int64

In [31]:
df_change.info()
df_change.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302104 entries, 0 to 302103
Data columns (total 5 columns):
EID        302104 non-null int64
ALTERNO    302104 non-null object
ALTDATE    302104 non-null object
ALTBE      37519 non-null object
ALTAF      37537 non-null object
dtypes: int64(1), object(4)
memory usage: 11.5+ MB


,EID,ALTERNO,ALTDATE,ALTBE,ALTAF
0,350,12,2015-02,NaN,NaN
1,399,05,2014-01,10,50
2,399,12,2015-05,NaN,NaN
3,399,12,2013-12,NaN,NaN
4,399,27,2014-01,10,50


In [32]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255130 entries, 0 to 255129
Data columns (total 32 columns):
EID                    255130 non-null int64
RGYEAR                 255130 non-null object
HY                     255130 non-null int64
ZCZB                   255130 non-null float64
ETYPE                  255130 non-null int64
MPNUM                  255130 non-null float64
INUM                   255130 non-null float64
FINZB                  255130 non-null float64
FSTINUM                255130 non-null float64
TZINUM                 255130 non-null float64
HY_ZCZB_AVG            255130 non-null float64
HY_ZCZB_AVG_RATE       255130 non-null float64
MPNUM_HY_AVG           255130 non-null float64
INUM_HY_AVG            255130 non-null float64
FINZB_HY_AVG           255130 non-null float64
FSTINUM_HY_AVG         255130 non-null float64
TZINUM_HY_AVG          255130 non-null float64
ETYPE_ZCZB_AVG         255130 non-null float64
ETYPE_ZCZB_AVG_RATE    255130 non-null float64
RGY

In [4]:
df_change = df_change.sort_values(['EID','ALTDATE'])


In [5]:
df_change = df_change.fillna('0')
columns = ['EID', 'ALTERNO', 'ALTDATE', 'ALTBE', 'ALTAF']
df_xchange = pd.DataFrame(columns=['EID', 'ALTERNO', 'ALTDATE', 'ALTBE', 'ALTAF'])

In [6]:
EIDS = set(df_change['EID'])

len(EIDS)

96967

In [7]:
k = 0
for EID in EIDS:
    if k%3000 == 0:
        print('第%d次处理--------->',k)
    k+=1
    tmp = df_change[df_change['EID'] == EID]
    row = [EID,tmp['ALTERNO'].values,tmp['ALTDATE'].values,tmp['ALTBE'].values,tmp['ALTAF'].values]
    
    df_xchange = df_xchange.append(pd.Series(row,columns),ignore_index=True)
    

第%d次处理---------> 0
第%d次处理---------> 3000
第%d次处理---------> 6000
第%d次处理---------> 9000
第%d次处理---------> 12000
第%d次处理---------> 15000
第%d次处理---------> 18000
第%d次处理---------> 21000
第%d次处理---------> 24000
第%d次处理---------> 27000
第%d次处理---------> 30000
第%d次处理---------> 33000
第%d次处理---------> 36000
第%d次处理---------> 39000
第%d次处理---------> 42000
第%d次处理---------> 45000
第%d次处理---------> 48000
第%d次处理---------> 51000
第%d次处理---------> 54000
第%d次处理---------> 57000
第%d次处理---------> 60000
第%d次处理---------> 63000
第%d次处理---------> 66000
第%d次处理---------> 69000
第%d次处理---------> 72000
第%d次处理---------> 75000
第%d次处理---------> 78000
第%d次处理---------> 81000
第%d次处理---------> 84000
第%d次处理---------> 87000
第%d次处理---------> 90000
第%d次处理---------> 93000
第%d次处理---------> 96000


In [8]:
df_xchange.info()
df_xchange.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96967 entries, 0 to 96966
Data columns (total 5 columns):
EID        96967 non-null float64
ALTERNO    96967 non-null object
ALTDATE    96967 non-null object
ALTBE      96967 non-null object
ALTAF      96967 non-null object
dtypes: float64(1), object(4)
memory usage: 3.7+ MB


,EID,ALTERNO,ALTDATE,ALTBE,ALTAF
0,524290.0,"[A_015, 03, A_015, 03, A_015, A_015]","[2013-09, 2013-09, 2013-09, 2013-09, 2014-02, ...","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
1,5.0,[12],[2015-04],[0],[0]
2,524293.0,[12],[2013-11],[0],[0]
3,262153.0,"[04, 10, 05]","[2014-12, 2014-12, 2014-12]","[0, 0, 50]","[0, 0, 200]"
4,262154.0,"[99, 12]","[2013-11, 2013-11]","[0, 0]","[0, 0]"


In [9]:
set(df_change['ALTERNO'])

{'01', '02', '03', '04', '05', '10', '12', '13', '14', '27', '99', 'A_015'}

In [10]:
def getFAlterno(x):
    m = x[0]
    if m == 'A_015':
        m = m[3:]
    return m

def getEAlterno(x):
    m = x[-1]
    if m == 'A_015':
        m = m[3:]
    return m

def getMAlterno(x):
    x = list(x)
    m = x[0]
    k = 0
    for i in x:
        if k<x.count(i):
            k=x.count(i)
            m = i
    if m == 'A_015':
        m = m[3:]
    return m


df_xchange['CHANGE_FIRST_ALTERNO'] = df_xchange['ALTERNO'].apply(getFAlterno)
df_xchange['CHANGE_END_ALTERNO'] = df_xchange['ALTERNO'].apply(getEAlterno)
df_xchange['CHANGE_ALTERNO_MUCNID'] = df_xchange['ALTERNO'].apply(getMAlterno)


In [11]:
df_xchange.head()

,EID,ALTERNO,ALTDATE,ALTBE,ALTAF,CHANGE_FIRST_ALTERNO,CHANGE_END_ALTERNO,CHANGE_ALTERNO_MUCNID
0,524290.0,"[A_015, 03, A_015, 03, A_015, A_015]","[2013-09, 2013-09, 2013-09, 2013-09, 2014-02, ...","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]",15,15,15
1,5.0,[12],[2015-04],[0],[0],12,12,12
2,524293.0,[12],[2013-11],[0],[0],12,12,12
3,262153.0,"[04, 10, 05]","[2014-12, 2014-12, 2014-12]","[0, 0, 50]","[0, 0, 200]",04,05,04
4,262154.0,"[99, 12]","[2013-11, 2013-11]","[0, 0]","[0, 0]",99,12,99


In [12]:
k = len(set(df_change['ALTERNO']))

In [13]:
def totalcnt(x):
    return len(x)

df_xchange['ALTERNO_SUM'] = df_xchange['ALTERNO'].apply(totalcnt)

def cnt(x):
    return len(set(x))

df_xchange['ALTERNO_CNT'] = df_xchange['ALTERNO'].apply(cnt)

df_xchange['ALTERNO_CNT_RATE'] = df_xchange['ALTERNO_CNT'] / k

def getFirst(x):
    x.sort()
    return x[0]

df_xchange['FIRST_CHANGE_TIME'] = df_xchange['ALTDATE'].apply(getFirst)

def getEnd(x):
    x.sort()
    return x[-1]

df_xchange['END_CHANGE_TIME'] = df_xchange['ALTDATE'].apply(getEnd)

def equal(x):
    ok = 0
    for d in x:
        if d[:d.find(":")] != d[d.find(":")+1:]:
            ok = 1
    return ok
df_xchange['IS_CHANGE'] = (df_xchange['ALTBE'] +':'+ df_xchange['ALTAF']).apply(equal)

In [13]:
df_xchange.to_csv('../data/public/2alter_1.csv',index=False,index_label=False)

# df_xchange = pd.read_csv('../data/public/2alter_1.csv')

In [33]:
df_all = pd.merge(df_all,df_xchange[['EID', 'ALTERNO_SUM','ALTERNO_CNT', 'ALTERNO_CNT_RATE', 'FIRST_CHANGE_TIME',
                            'END_CHANGE_TIME', 'IS_CHANGE','CHANGE_FIRST_ALTERNO','CHANGE_END_ALTERNO','CHANGE_ALTERNO_MUCNID']],how='left',on=['EID'])

In [34]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 41 columns):
EID                      255130 non-null int64
RGYEAR                   255130 non-null object
HY                       255130 non-null int64
ZCZB                     255130 non-null float64
ETYPE                    255130 non-null int64
MPNUM                    255130 non-null float64
INUM                     255130 non-null float64
FINZB                    255130 non-null float64
FSTINUM                  255130 non-null float64
TZINUM                   255130 non-null float64
HY_ZCZB_AVG              255130 non-null float64
HY_ZCZB_AVG_RATE         255130 non-null float64
MPNUM_HY_AVG             255130 non-null float64
INUM_HY_AVG              255130 non-null float64
FINZB_HY_AVG             255130 non-null float64
FSTINUM_HY_AVG           255130 non-null float64
TZINUM_HY_AVG            255130 non-null float64
ETYPE_ZCZB_AVG           255130 non-null float64
ETYPE_ZCZB_AVG

,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,TZINUM_HY_RATE,ALTERNO_SUM,ALTERNO_CNT,ALTERNO_CNT_RATE,FIRST_CHANGE_TIME,END_CHANGE_TIME,IS_CHANGE,CHANGE_FIRST_ALTERNO,CHANGE_END_ALTERNO,CHANGE_ALTERNO_MUCNID
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> 空值填充，根据HY的类别的平均值或者众数进行填充
   + ['ALTERNO_SUM','ALTERNO_CNT', 'ALTERNO_CNT_RATE',]使用同一个大类别的均值进行填充
   + ['FIRST_CHANGE_TIME','END_CHANGE_TIME', 'IS_CHANGE' ]使用同一个大类别的众数进行填充



In [35]:
HYLIST = set(df_all['HY'])

print(HYLIST)

meanlist = ['ALTERNO_SUM','ALTERNO_CNT', 'ALTERNO_CNT_RATE']
modelist = ['FIRST_CHANGE_TIME','END_CHANGE_TIME', 'IS_CHANGE','CHANGE_FIRST_ALTERNO','CHANGE_END_ALTERNO','CHANGE_ALTERNO_MUCNID' ]


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 93}


In [36]:

int(df_all[df_all['HY']==75]['ALTERNO_SUM'].mean())

df_all[df_all['HY']==7]['CHANGE_FIRST_ALTERNO'].value_counts().index[0]



'02'

In [37]:
for HY in HYLIST:
    # print(df_train['HY'].value_counts())
    for d in meanlist:
        df_all.loc[df_all[df_all[d].isnull()][df_all['HY']==HY].index,d] = df_all[df_all['HY']==HY][d].mean()
    
    for c in modelist:
        if(len(df_all[df_all['HY']==HY][c].value_counts().index)==0): continue
        df_all.loc[df_all[df_all[c].isnull()][df_all['HY']==HY].index,c] = df_all[df_all['HY']==HY][c].value_counts().index[0]


/home/xiaoran/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/xiaoran/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [23]:
# df_all[df_all['FIRST_CHANGE_TIME'].isnull()]['FIRST_CHANGE_TIME'] = df_all[df_all['FIRST_CHANGE_TIME'].isnull()]['RGYEAR'] 

# df_all.loc[df_all['FIRST_CHANGE_TIME'].isnull(),'FIRST_CHANGE_TIME'] = df_all.loc[df_all['FIRST_CHANGE_TIME'].isnull(),'RGYEAR']
# df_all.loc[df_all['END_CHANGE_TIME'].isnull(),'END_CHANGE_TIME'] = df_all.loc[df_all['END_CHANGE_TIME'].isnull(),'RGYEAR']

In [38]:
def timeDiff(x):
    a = x[:x.find(':')]
    b = x[x.find(':')+1:]
    y = int(a[:a.find('-')]) - int(b[:b.find('-')])
    m = int(a[a.find('-')+1:]) - int(b[b.find('-')+1:])
    return y * 12 + m
    
df_all['FIRST_CHANGE_TIME_DIFF'] = (df_all['FIRST_CHANGE_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)
df_all['END_CHANGE_TIME_DIFF'] = (df_all['END_CHANGE_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)

df_all['END_FIRST_CHANGE_TIME_DIFF'] = (df_all['END_CHANGE_TIME'] + ':' + df_all['FIRST_CHANGE_TIME']).apply(timeDiff)


In [41]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 46 columns):
EID                           255130 non-null int64
RGYEAR                        255130 non-null object
HY                            255130 non-null int64
ZCZB                          255130 non-null float64
ETYPE                         255130 non-null int64
MPNUM                         255130 non-null float64
INUM                          255130 non-null float64
FINZB                         255130 non-null float64
FSTINUM                       255130 non-null float64
TZINUM                        255130 non-null float64
HY_ZCZB_AVG                   255130 non-null float64
HY_ZCZB_AVG_RATE              255130 non-null float64
MPNUM_HY_AVG                  255130 non-null float64
INUM_HY_AVG                   255130 non-null float64
FINZB_HY_AVG                  255130 non-null float64
FSTINUM_HY_AVG                255130 non-null float64
TZINUM_HY_AVG                 25

In [40]:
df_all['CHANGE_PRE_CNT'] = df_all['ALTERNO_SUM'] / df_all['END_CHANGE_TIME_DIFF']
df_all['CHANGE_CNT_RATE'] = df_all['ALTERNO_SUM'] / df_all['ALTERNO_SUM'].mean()


In [42]:
tmp = pd.DataFrame()

tmp['CHANGE_HY_CNT_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['ALTERNO_SUM'].mean()
tmp['CHANGE_HY_CNT_ALL_AVG'] = tmp['CHANGE_HY_CNT_AVG'] / df_all['ALTERNO_SUM'].mean()
tmp['HY'] = tmp.index





In [45]:
df_all1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 48 columns):
EID                           255130 non-null int64
RGYEAR                        255130 non-null object
HY                            255130 non-null int64
ZCZB                          255130 non-null float64
ETYPE                         255130 non-null int64
MPNUM                         255130 non-null float64
INUM                          255130 non-null float64
FINZB                         255130 non-null float64
FSTINUM                       255130 non-null float64
TZINUM                        255130 non-null float64
HY_ZCZB_AVG                   255130 non-null float64
HY_ZCZB_AVG_RATE              255130 non-null float64
MPNUM_HY_AVG                  255130 non-null float64
INUM_HY_AVG                   255130 non-null float64
FINZB_HY_AVG                  255130 non-null float64
FSTINUM_HY_AVG                255130 non-null float64
TZINUM_HY_AVG                 25

In [46]:
df_all = pd.merge(df_all,tmp,how='left',on=['HY'])


In [47]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 48 columns):
EID                           255130 non-null int64
RGYEAR                        255130 non-null object
HY                            255130 non-null int64
ZCZB                          255130 non-null float64
ETYPE                         255130 non-null int64
MPNUM                         255130 non-null float64
INUM                          255130 non-null float64
FINZB                         255130 non-null float64
FSTINUM                       255130 non-null float64
TZINUM                        255130 non-null float64
HY_ZCZB_AVG                   255130 non-null float64
HY_ZCZB_AVG_RATE              255130 non-null float64
MPNUM_HY_AVG                  255130 non-null float64
INUM_HY_AVG                   255130 non-null float64
FINZB_HY_AVG                  255130 non-null float64
FSTINUM_HY_AVG                255130 non-null float64
TZINUM_HY_AVG                 25

,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,CHANGE_FIRST_ALTERNO,CHANGE_END_ALTERNO,CHANGE_ALTERNO_MUCNID,FIRST_CHANGE_TIME_DIFF,END_CHANGE_TIME_DIFF,END_FIRST_CHANGE_TIME_DIFF,CHANGE_PRE_CNT,CHANGE_CNT_RATE,CHANGE_HY_CNT_AVG,CHANGE_HY_CNT_ALL_AVG
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,12,12,12,182,182,0,0.015744,0.924004,2.865397,0.924004
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,12,12,12,146,146,0,0.020849,0.981587,3.043965,0.981587
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,12,12,12,26,26,0,0.124123,1.040675,3.227200,1.040675
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,12,12,12,14,14,0,0.230514,1.040675,3.227200,1.040675
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,12,12,12,86,86,0,0.037526,1.040675,3.227200,1.040675


In [48]:
df_all.to_csv('../data/alldata/df_data12.csv',index=False,index_label=False)

In [30]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 24 columns):
EID                           255130 non-null object
RGYEAR                        255130 non-null object
RGYEAR_DIFF                   255130 non-null int64
ZCZB                          255130 non-null float64
HY                            255130 non-null int64
ETYPE                         255130 non-null int64
ZCZB_RATE                     255130 non-null float64
ETYPE_RATE                    255130 non-null float64
MPNUM                         255130 non-null float64
INUM                          255130 non-null float64
FINZB                         255130 non-null float64
FSTINUM                       255130 non-null float64
TZINUM                        255130 non-null float64
ALTERNO_SUM                   255130 non-null float64
ALTERNO_CNT                   255130 non-null float64
ALTERNO_CNT_RATE              255130 non-null float64
FIRST_CHANGE_TIME             255

In [39]:
# 得到在df_xchange的所有数据，
# df_xall = df_all[df_all['EID'].isin(df_xchange['EID'])]


96967

In [42]:
# 分成训练和测试文件写入文件
# df_train,df_test = pro_data.spliteData(df_xall)
# df_train.to_csv('../data/newData/df_train01.csv',index=False,index_label=False)
# df_test.to_csv('../data/newData/df_test01.csv',index=False,index_label=False)

# df_all.to_csv('../data/alldata/df_data01.csv',index=False,index_label=False)